<a href="https://colab.research.google.com/github/leon3108/Applied/blob/main/NLP_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing </a>

## Assignment: K Nearest Neighbors Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

You can follow these steps:
1. Read training-test data (Given)
2. Train a KNN classifier (Implement)
3. Make predictions on your test dataset (Implement)

__You can use the KNN Classifier from here: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html__

## 1. Reading the dataset

We will use the __pandas__ library to read our dataset.

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [3]:
import pandas as pd

train_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


#### __Test data:__

In [4]:
import pandas as pd

test_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


## 2. Train a KNN Classifier
Here, you will apply pre-processing operations we covered in the class. Then, you can split your dataset to training and validation here. For your first submission, you will use __K Nearest Neighbors Classifier__. It is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

In [5]:
import nltk, re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Implement this
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

stop = stopwords.words('english')

excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts):
  final_text_list=[]
  for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent)

        for w in word_tokenize(sent):
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence)

        final_text_list.append(final_string)

  return final_text_list

In [7]:
print("Process text")
train_text_list = process_text(train_df["text"])
val_text_list = process_text(test_df["text"])

Process text


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    # ('text_vect', CountVectorizer(binary = True,
    ( 'text_vect', TfidfVectorizer(binary=True,
                                  max_features=100)),
    ('knn', KNeighborsClassifier(10))
                                ])


In [14]:
# We using lists of processed text fields
X_train = train_text_list
X_val = val_text_list

y_train = train_df["label"]
y_val = test_df["label"]

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', TfidfVectorizer(binary=True, max_features=100)),
                ('knn', KNeighborsClassifier(n_neighbors=10))])

## 3. Make predictions on your test dataset

Once we select our best performing model, we can use it to make predictions on the test dataset. You can simply use __.fit()__ function with your training data to use the best performing K value and use __.predict()__ with your test data to get your test predictions.

In [15]:
# Implement this
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[9373 3127]
 [5149 7351]]
              precision    recall  f1-score   support

           0       0.65      0.75      0.69     12500
           1       0.70      0.59      0.64     12500

    accuracy                           0.67     25000
   macro avg       0.67      0.67      0.67     25000
weighted avg       0.67      0.67      0.67     25000

Accuracy (validation): 0.66896


<table align="center">
	<thead>
		<tr>
			<th colspan="1"><b>CountVectorizer</b></th>
			<th colspan="1"><b>CountVectorizer with binary</b></th>
			<th colspan="1"><b>TfidfVectorizer</b></th>
			<th colspan="1"><b>TfidfVectorizer with binary</b></th>
			<th colspan="1"><b>max_features</b></th>
			<th colspan="1"><b>k</b></th>
			<th colspan="1"><b>results</b></th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">10</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.53</td><td>0.64</td><td>0.58</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.54</td><td>0.43</td><td>0.48</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.53</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.53</td><td>0.53</td><td>0.53</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.53</td><td>0.53</td><td>0.53</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.5316</caption>
                </table>
            </td>
		</tr>
    	<tr>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">50</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.62</td><td>0.72</td><td>0.67</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.67</td><td>0.57</td><td>0.61</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.64</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.64</td><td>0.64</td><td>0.64</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.64</td><td>0.64</td><td>0.64</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.64104</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">50</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.63</td><td>0.62</td><td>0.62</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.63</td><td>0.63</td><td>0.63</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.63</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.63</td><td>0.63</td><td>0.63</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.63</td><td>0.63</td><td>0.63</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.62652</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">10</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.52</td><td>0.54</td><td>0.53</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.52</td><td>0.50</td><td>0.51</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.52</td><td>0.52</td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.52</td><td>0.52</td><td>0.52</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.52288</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">100</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.64</td><td>0.63</td><td>0.63</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.64</td><td>0.65</td><td>0.64</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.64</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.64</td><td>0.64</td><td>0.64</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.64</td><td>0.64</td><td>0.64</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.63816</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">10</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.52</td><td>0.67</td><td>0.59</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.54</td><td>0.38</td><td>0.45</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.53</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.53</td><td>0.53</td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.53</td><td>0.53</td><td>0.52</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.52652</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">100</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.64</td><td>0.73</td><td>0.68</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.68</td><td>0.59</td><td>0.63</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.66</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.66</td><td>0.66</td><td>0.66</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.66</td><td>0.66</td><td>0.66</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.65788</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center">10</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.53</td><td>0.53</td><td>0.53</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.53</td><td>0.52</td><td>0.52</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.53</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.53</td><td>0.53</td><td>0.53</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.53</td><td>0.53</td><td>0.53</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.52684</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center">100</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.67</td><td>0.68</td><td>0.67</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.67</td><td>0.66</td><td>0.67</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.67</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.67</td><td>0.67</td><td>0.67</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.67</td><td>0.67</td><td>0.67</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.66948</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center">10</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.53</td><td>0.65</td><td>0.58</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.55</td><td>0.42</td><td>0.48</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.54</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.54</td><td>0.54</td><td>0.53</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.54</td><td>0.54</td><td>0.53</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.53548</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center"></td>
			<td align="center">100</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.66</td><td>0.76</td><td>0.70</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.71</td><td>0.60</td><td>0.65</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.68</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.68</td><td>0.68</td><td>0.68</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.68</td><td>0.68</td><td>0.68</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.67924</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center">10</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.52</td><td>0.53</td><td>0.52</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.52</td><td>0.51</td><td>0.52</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.52</td><td>0.52</td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.52</td><td>0.52</td><td>0.52</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.51984</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center">100</td>
			<td align="center">5</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.65</td><td>0.66</td><td>0.65</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.65</td><td>0.65</td><td>0.65</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.65</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.65</td><td>0.65</td><td>0.65</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.65</td><td>0.65</td><td>0.65</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.65244</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center">10</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.52</td><td>0.67</td><td>0.59</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.53</td><td>0.38</td><td>0.44</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.52</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.53</td><td>0.52</td><td>0.51</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.53</td><td>0.52</td><td>0.51</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.52436</caption>
                </table>
            </td>
		</tr>
    <tr>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center"></td>
			<td align="center">x</td>
			<td align="center">100</td>
			<td align="center">10</td>
			<td align="center">
                <table>
                    <tr>
                      <td></td><th scope="col">precision</th>
                      <th scope="col">recall</th>
                      <th scope="col">f1-score</th>
                      <th scope="col">support</th>
                    </tr>
                    <tr>
                      <th scope="row">0</th><td>0.65</td><td>0.75</td><td>0.69</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">1</th><td>0.70</td><td>0.59</td><td>0.64</td><td>12500</td>
                    </tr>
                    <tr>
                      <th scope="row">Accuracy</th><td></td><td></td><td>0.67</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">Macro avg</th><td>0.67</td><td>0.67</td><td>0.67</td><td>25000</td>
                    </tr>
                    <tr>
                      <th scope="row">weighted avg</th><td>0.67</td><td>0.67</td><td>0.67</td><td>25000</td>
                    </tr>
                    <caption>Accuracy (Validation) : 0.66896</caption>
                </table>
            </td>
		</tr>
    </tbody>
</table>